In [1]:
import random 
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from shapely import wkt 
from shapely.geometry import Point

In [2]:
### node file
nodes_df = pd.read_csv('../network_inputs/osm_nodes.csv')
nodes_gdf = gpd.GeoDataFrame(nodes_df, geometry=[Point(xy) for xy in zip(nodes_df.lon, nodes_df.lat)], crs='epsg:4326').to_crs('epsg:3857')
nodes_df.head()

### city boundary
cities = gpd.read_file('../network_inputs/ca-places-boundary/CA_Places_TIGER2016.shp').to_crs('epsg:3857')
### individual cities
paradise = cities[cities['NAMELSAD']=='Paradise town'].copy()
magalia = cities[cities['NAMELSAD']=='Magalia CDP'].copy()
chico = cities[cities['NAMELSAD']=='Chico city'].copy()
oroville = cities.loc[cities['NAME'].isin(['Palermo', 'Oroville East', 'South Oroville', 'Kelly Ridge', 'Oroville', 'Thermalito'])].copy()

### parcel file
parcels = gpd.read_file('BCAG_GIS_Request/parcel/parcel.shp').to_crs(3857)

### evacuation zone
evac_zones = gpd.read_file('evacuation_zone/digitized_evacuation_zone/digitized_evacuation_zone.shp').to_crs(3857)

/Users/bingyu/anaconda3/envs/geo/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/bingyu/anaconda3/envs/geo/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [3]:
### Residential parcels in Paradise
### did not consider Magalia and needs to be changed in the future
### did not consider all land use types

### While the pre-fire population of Paradise was an estimated 26,800, according to the U.S. 2010 Census, a door-to-door survey conducted in April of this year revealed only 2,034 residents remain
### https://sf.curbed.com/2019/7/12/20692079/town-destroyed-by-pg-e-fire-loses-92-percent-of-its-population
paradise_parcels = parcels.loc[(parcels['JURISDICTI']=='Paradise') & (parcels['LANDUSE'].isin(['RZ', 'RV', 'RS']))].copy()
paradise_parcels.groupby('LANDUSE').size()

LANDUSE
RS    2655
RV     609
RZ    6811
dtype: int64

In [4]:
### Map residential parcels in Paradise to nodes
from scipy.spatial.distance import cdist 

paradise_parcels['centroid'] = paradise_parcels.apply(lambda x: x['geometry'].centroid, axis=1)
paradise_parcels = paradise_parcels.set_geometry('centroid')
paradise_parcels['c_x'] = paradise_parcels['centroid'].x
paradise_parcels['c_y'] = paradise_parcels['centroid'].y
nodes_xy = np.array([nodes_gdf['geometry'].x.values, nodes_gdf['geometry'].y.values]).transpose()
nodes_osmid = nodes_gdf['node_osmid'].values
def get_closest_node(parcel_x, parcel_y):
    return nodes_osmid[cdist([(parcel_x, parcel_y)], nodes_xy, 'euclidean').argmin()]
paradise_parcels['closest_node'] = paradise_parcels.apply(lambda x: get_closest_node(x['c_x'], x['c_y']), axis=1)
nodes_evac_origins = paradise_parcels.groupby('closest_node').size().reset_index().rename(columns={'closest_node': 'node_id', 0: 'hh_cnts'})
nodes_evac_origins.head()

,node_id,hh_cnts
0,86370139,2
1,86370151,16
2,86370157,7
3,86370172,4
4,86370389,5


In [5]:
### assign an evacuation zone to each evacuation origin
nodes_evac_origins_gdf = nodes_gdf.merge(nodes_evac_origins, how='right', right_on='node_id', left_on='node_osmid')
nodes_evac_origins_sindex = nodes_evac_origins_gdf.sindex
nodes_evac_origins_gdf['evac_zone'] = -1

for zone in evac_zones.itertuples():
    zone_id = getattr(zone, 'id')
    zone_geom = getattr(zone, 'geometry')
    coarse_node_ids = list(nodes_evac_origins_sindex.intersection(zone_geom.bounds))
    coarse_nodes = nodes_evac_origins_gdf.iloc[coarse_node_ids]
    precise_nodes = coarse_nodes[coarse_nodes.intersects(zone_geom)]
    nodes_evac_origins_gdf['evac_zone'] = np.where(nodes_evac_origins_gdf['node_osmid'].isin(
        precise_nodes['node_osmid']), zone_id, nodes_evac_origins_gdf['evac_zone'])
nodes_evac_origins_gdf.groupby('evac_zone').agg({'hh_cnts': np.sum})

,hh_cnts
evac_zone,
-1,82
1,1099
2,1056
3,863
4,1148
5,674
6,735
7,970
8,427


In [11]:
### make OD
### only look at 80% of evacuee whose destination is at 80%
### assume 2 veh per household
od = nodes_evac_origins_gdf[['node_id_igraph', 'node_osmid', 'hh_cnts', 'evac_zone']].copy().sample(frac=0.8)
od = od.loc[od.index.repeat(od.hh_cnts*2)].reset_index(drop=True)
od['origin_osmid'] = od['node_osmid']
od['one_destin_osmid'] = 86375792
# od[['origin_osmid', 'one_destin_osmid', 'evac_zone']].to_csv('od.csv', index=False)
print(od.shape, nodes_evac_origins_gdf.shape)
od.tail()

(16106, 6) (1739, 9)


,node_id_igraph,node_osmid,hh_cnts,evac_zone,origin_osmid,one_destin_osmid
16101,1570,86543631,9,4,86543631,86375792
16102,1570,86543631,9,4,86543631,86375792
16103,1570,86543631,9,4,86543631,86375792
16104,1570,86543631,9,4,86543631,86375792
16105,1570,86543631,9,4,86543631,86375792


In [12]:
### If destination is Oroville or Chico
print(oroville['NAME'].values.tolist()) ### Should be 6
safe_area_dict = {'chico': chico.unary_union, 'oroville': oroville.unary_union}
safe_nodes = dict()
nodes_sindex = nodes_gdf.sindex
for nm, geom in safe_area_dict.items():
    coarse_node_ids = list(nodes_sindex.intersection(geom.bounds))
    coarse_nodes = nodes_gdf.iloc[coarse_node_ids]
    precise_nodes = coarse_nodes[coarse_nodes.intersects(geom)]
    safe_nodes[nm] = precise_nodes['node_osmid'].values.tolist()
    print(nm, len(precise_nodes))

chico_ratio = 0.7
d_list = random.choices(safe_nodes['chico'], k=int(od.shape[0]*chico_ratio))
d_list += random.choices(safe_nodes['oroville'], k=od.shape[0]-len(d_list))
od['destin_osmid'] = d_list

### add departure time
od = od.loc[od['evac_zone']<=14]
od['dept_time_scen_1'] = 0
od['dept_time_scen_2'] = np.where(
    od['evac_zone'].isin([1,2,3]), 0, np.where(
        od['evac_zone'].isin([4,5,6,7,8]), 157*60, np.where(
            od['evac_zone'].isin([9,10,11,12,13,14]), (157+205)*60, 0
    )))

od[['origin_osmid', 'one_destin_osmid', 'destin_osmid', 'evac_zone', 'dept_time_scen_1', 'dept_time_scen_2']].to_csv('od.csv', index=False)

['Oroville', 'Kelly Ridge', 'Oroville East', 'Palermo', 'South Oroville', 'Thermalito']
chico 3212
oroville 2507
